# Multi-Agent Systems with Strands Agents - Part 2: Peer-to-Peer Collaboration using Swarm

## 🎯 Overview

This notebook demonstrates the **second paradigm** of multi-agent collaboration: **Peer-to-Peer Collaboration using Swarm**. 

In this pattern, agents communicate with each other as equals, sharing information and working together without a central coordinator. Unlike the hierarchical "Agent as Tools" pattern, swarm agents collaborate in a more distributed and flexible manner.

## 🏗️ Architecture Pattern: Swarm

```
┌─────────────────────────────────────────────────────────────┐
│                    Swarm Coordinator                        │
│                    (Uses swarm tool)                        │
└────────────────────────────┬────────────────────────────────┘
                             │
                             ▼
┌─────────────────────────────────────────────────────────────┐
│                   Agent Swarm Network                       │
│                                                             │
│  ┌─────────────┐    ┌─────────────┐    ┌─────────────┐      │
│  │   Agent A   │◄──►│   Agent B   │◄──►│   Agent C   │      │
│  │             │    │             │    │             │      │
│  └─────────────┘    └─────────────┘    └─────────────┘      │
│         ▲                  ▲                  ▲             │
│         │                  │                  │             │
│         └──────────────────┼──────────────────┘             │
│                            │                                │
│                       Peer-to-Peer                          │
│                      Communication                          │
└─────────────────────────────────────────────────────────────┘
```

## 🔑 Key Features

- **Distributed Collaboration**: No single point of control
- **Peer Communication**: Agents can directly communicate with each other
- **Iterative Improvement**: Multiple rounds of collaboration and refinement
- **Flexible Workflows**: Dynamic interaction patterns based on task needs
- **Collective Intelligence**: Leveraging diverse agent capabilities

## 📋 What We'll Build

We'll implement several practical examples:
1. **Collaborative Content Creation** - Writer, Editor, Fact-checker swarm
2. **Product Development Team** - Designer, Developer, Tester swarm
3. **Research Analysis Team** - Researcher, Analyst, Reviewer swarm
4. **Marketing Campaign Team** - Strategist, Creative, Analyst swarm

---

## 🛠️ Setup and Installation

First, let's install the required packages and verify our setup:

In [1]:
# Install required packages
!pip install -U strands-agents strands-agents-tools boto3

  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached typing_inspection-0.4.1-py3-none-any.whl.metadata (2.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 159.0 MB/s eta 0:00:00
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached httpcore-1.0.9-py3-none-any.whl (78 kB)
Using cached typing_inspection-0.4.1-py3-none-any.whl (14 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 110.4 MB/s eta 0:00:00
  Attempting uninstall: botocorem╸━━━━━━━━━━━━━━━━━━━━━━━ 11/26 [opentelemetry-api]
    Found existing installation: botocore 1.39.13━━━━━━━━━━━━━ 11/26 [opentelemetry-api]
    Uninstalling botocore-1.39.13:90m╺━━━━━━━━━━━━━━━━━━━ 13/26 [botocore]-api]
      Successfully uninstalled botocore-1.39.13━━━━━━━━━━━━━━━ 13/26 [botocore]
  Attempting uninstall: boto3━━━━━━━━━━━╺━━━━━━━ 21/26 [mcp]x]quests-auth]
    Found existing installation: boto3 1.39.13━━━━━━━ 21/26 [mcp]
 

In [2]:
# Import required libraries
import boto3
import json
from datetime import datetime
from typing import Dict, List, Any

# Strands Agents imports
from strands import Agent, tool
from strands_tools import swarm

print("📦 All imports successful!")

📦 All imports successful!


In [3]:
# Verify installation and test basic functionality
def verify_installation():
    """Verify that all required packages are installed correctly."""
    try:
        # Test imports
        from strands import Agent
        from strands_tools import swarm
        print("✅ All required imports successful!")
        
        # Test basic agent creation (without calling the model)
        test_agent = Agent(
            model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
            system_prompt="You are a test agent."
        )
        print("✅ Claude 3.7 Sonnet agent creation successful!")
        print("\n🎉 Installation verified - ready to proceed with swarm collaboration!")
        
    except Exception as e:
        print(f"❌ Installation verification failed: {e}")
        print("Please ensure you have installed: pip install -U strands-agents strands-agents-tools boto3")

verify_installation()

✅ All required imports successful!
✅ Claude 3.7 Sonnet agent creation successful!

🎉 Installation verified - ready to proceed with swarm collaboration!


## ⚙️ Configuration

Let's configure our AWS Bedrock settings and Claude 3.7 Sonnet model:

In [4]:
# Configure AWS region for Bedrock
AWS_REGION = "us-east-1"  # Claude 3.7 Sonnet is available in us-east-1
CLAUDE_MODEL_ID = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"

# Verify AWS credentials are configured
try:
    session = boto3.Session()
    credentials = session.get_credentials()
    if credentials is None:
        print("❌ AWS credentials not found. Please configure your AWS credentials.")
        print("Run: aws configure")
    else:
        print("✅ AWS credentials configured successfully")
        print(f"✅ Using region: {AWS_REGION}")
        print(f"✅ Using model: {CLAUDE_MODEL_ID}")
except Exception as e:
    print(f"❌ AWS configuration error: {e}")

✅ AWS credentials configured successfully
✅ Using region: us-east-1
✅ Using model: us.anthropic.claude-3-7-sonnet-20250219-v1:0


## 📝 Example 1: Collaborative Content Creation Swarm

Our first example demonstrates a content creation swarm with three specialized agents:
- **Content Writer**: Creates engaging and informative content
- **Editor**: Improves grammar, clarity, and structure
- **Fact-Checker**: Verifies claims and identifies potential errors

In [5]:
# Create the swarm coordinator agent
content_swarm_coordinator = Agent(
    model=CLAUDE_MODEL_ID,
    tools=[swarm],
    system_prompt="""You are a collaboration coordinator that facilitates peer-to-peer agent swarms for content creation.
    
    Use the swarm tool to create networks of agents that can work together on collaborative content creation tasks.
    
    Key principles:
    - Ensure each agent has a clear, specialized role
    - Facilitate iterative improvement through multiple collaboration rounds
    - Encourage peer-to-peer feedback and refinement
    - Aim for high-quality, well-researched, and engaging final output
    """
)

def collaborative_content_creation(topic: str, target_audience: str = "general audience") -> str:
    """Create content collaboratively using a swarm of specialized agents.
    
    Args:
        topic: The topic to write about
        target_audience: The intended audience for the content
    
    Returns:
        The final collaborative content
    """
    swarm_request = f"""
    Create a swarm of three agents to collaboratively develop high-quality content about: {topic}
    Target audience: {target_audience}
    
    The swarm should include:
    1. A creative content writer who:
       - Creates engaging and informative content
       - Uses compelling storytelling techniques
       - Adapts tone and style for the target audience
       - Focuses on clarity and readability
    
    2. An experienced editor who:
       - Improves content by fixing grammar and syntax
       - Enhances clarity and flow
       - Improves structure and organization
       - Suggests better word choices and phrasing
    
    3. A meticulous fact-checker who:
       - Identifies potential factual errors
       - Verifies claims and statistics
       - Suggests additional credible sources
       - Ensures accuracy and reliability
    
    Collaboration Process:
    - The content writer creates an initial draft
    - The editor reviews and improves the draft
    - The fact-checker verifies accuracy and suggests improvements
    - All agents collaborate for 2 rounds of iterative improvements
    - The content writer incorporates all feedback for the final version
    
    Please ensure the final article is:
    - Well-structured with clear sections
    - Engaging and informative
    - Factually accurate
    - Appropriate for the target audience
    - Between 800-1200 words
    """
    
    print(f"🚀 Starting collaborative content creation for: {topic}")
    print(f"👥 Target audience: {target_audience}")
    print("⏳ This may take a few moments as agents collaborate...\n")
    
    result = content_swarm_coordinator(swarm_request)
    return result

print("✅ Content creation swarm coordinator ready!")

✅ Content creation swarm coordinator ready!


In [6]:
# Test the collaborative content creation swarm
topic = "The Future of AI Agents in Business Automation"
audience = "business executives and technology leaders"

content_result = collaborative_content_creation(topic, audience)
print("📄 Final Collaborative Content:")
print("=" * 50)
print(content_result)

🚀 Starting collaborative content creation for: The Future of AI Agents in Business Automation
👥 Target audience: business executives and technology leaders
⏳ This may take a few moments as agents collaborate...

I'll create a swarm of three specialized agents to develop high-quality content about "The Future of AI Agents in Business Automation" for business executives and technology leaders. The swarm will include a creative content writer, an experienced editor, and a meticulous fact-checker who will collaborate through multiple rounds of feedback and improvement.

Let me set up this swarm with the collaboration process you outlined:
Tool #1: swarm
I'll create a comprehensive article on "The Future of AI Agents in Business Automation" for business executives and technology leaders. Let me start by crafting an engaging initial draft with compelling storytelling and clear structure.

# The Future of AI Agents in Business Automation: From Science Fiction to Strategic Necessity

In 2019, 

# The Future of AI Agents in Business Automation: Transforming Enterprise Operations

## Executive Summary

Artificial Intelligence (AI) agents are poised to revolutionize business automation over the next decade, representing a significant evolution beyond traditional automation tools. These intelligent systems can autonomously perform complex tasks, learn from interactions, and make decisions within defined parameters—all while working alongside human teams. For executive leadership, understanding this emerging technology is critical for maintaining competitive advantage and driving operational excellence.

## The Evolution from Basic Automation to AI Agents

Traditional automation has long focused on executing predefined, repetitive tasks through rigid programming. The journey from simple scripts to today's AI agents marks a profound transformation in business technology:

- **First wave (1990s-2000s)**: Basic process automation with limited flexibility
- **Second wave (2010s)**: Ro

## 🛠️ Example 2: Product Development Team Swarm

Let's create a more complex swarm that simulates a product development team:
- **Product Designer**: Creates user experience and interface designs
- **Software Developer**: Implements technical solutions
- **Quality Tester**: Tests functionality and identifies issues

In [7]:
# Create product development swarm coordinator
product_swarm_coordinator = Agent(
    model=CLAUDE_MODEL_ID,
    tools=[swarm],
    system_prompt="""You are a product development coordinator that facilitates collaborative product development using agent swarms.
    
    Use the swarm tool to create teams of specialized agents that work together on product development tasks.
    
    Focus on:
    - User-centered design principles
    - Technical feasibility and best practices
    - Quality assurance and testing
    - Iterative improvement and refinement
    - Cross-functional collaboration
    """
)

def collaborative_product_development(product_idea: str, target_users: str, platform: str = "web application") -> str:
    """Develop a product concept collaboratively using a swarm of specialists.
    
    Args:
        product_idea: Description of the product to develop
        target_users: The intended users of the product
        platform: The target platform (web, mobile, desktop, etc.)
    
    Returns:
        Comprehensive product development plan
    """
    swarm_request = f"""
    Create a swarm of three product development specialists to collaboratively develop: {product_idea}
    Target users: {target_users}
    Platform: {platform}
    
    The development team should include:
    
    1. A UX/UI Product Designer who:
       - Analyzes user needs and pain points
       - Creates user personas and journey maps
       - Designs intuitive user interfaces
       - Focuses on accessibility and usability
       - Considers design systems and consistency
    
    2. A Senior Software Developer who:
       - Evaluates technical feasibility
       - Designs system architecture
       - Identifies required technologies and frameworks
       - Considers scalability and performance
       - Estimates development effort and timeline
    
    3. A Quality Assurance Tester who:
       - Identifies potential user experience issues
       - Plans testing strategies and scenarios
       - Considers edge cases and error handling
       - Evaluates security and privacy concerns
       - Suggests quality metrics and success criteria
    
    Collaboration Process:
    - Designer creates initial user research and design concepts
    - Developer reviews feasibility and suggests technical approaches
    - Tester identifies potential issues and testing requirements
    - All team members collaborate for 2 rounds of refinement
    - Final integrated product development plan is created
    
    Deliverables should include:
    - User research and personas
    - Feature specifications
    - Technical architecture overview
    - Development timeline and milestones
    - Testing strategy and quality metrics
    - Risk assessment and mitigation plans
    """
    
    print(f"🚀 Starting collaborative product development for: {product_idea}")
    print(f"👥 Target users: {target_users}")
    print(f"💻 Platform: {platform}")
    print("⏳ Development team is collaborating...\n")
    
    result = product_swarm_coordinator(swarm_request)
    return result

print("✅ Product development swarm coordinator ready!")

✅ Product development swarm coordinator ready!


In [8]:
# Test the product development swarm
product_idea = "A smart task management app that uses AI to prioritize tasks and suggest optimal work schedules"
target_users = "busy professionals and remote workers"
platform = "mobile application (iOS and Android)"

product_result = collaborative_product_development(product_idea, target_users, platform)
print("📋 Product Development Plan:")
print("=" * 50)
print(product_result)

🚀 Starting collaborative product development for: A smart task management app that uses AI to prioritize tasks and suggest optimal work schedules
👥 Target users: busy professionals and remote workers
💻 Platform: mobile application (iOS and Android)
⏳ Development team is collaborating...

I'll create a swarm of three product development specialists to collaborate on developing the smart task management app according to your specifications. Let me set that up for you right away.
Tool #1: swarm
I'll start by conducting comprehensive user research and developing the foundational design elements for this smart task management app. Let me begin with user research, personas, and journey mapping before collaborating with the other agents.

## Phase 1: User Research & Analysis

### Target User Research
**Primary Users: Busy Professionals & Remote Workers**

**Key Pain Points Identified:**
- Overwhelming task lists with unclear priorities
- Difficulty managing time across multiple projects
- Lac

cycle failed
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/strands/models/bedrock.py", line 431, in _stream
    response = self.client.converse_stream(**request)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/client.py", line 601, in _api_call
    return self._make_api_call(operation_name, kwargs)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/context.py", line 123, in wrapper
    return func(*args, **kwargs)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/client.py", line 1074, in _make_api_call
    raise error_class(parsed_response, operation_name)
botocore.errorfactory.ThrottlingException: An error occurred (ThrottlingException) when calling the ConverseStream operation (reached max retries: 4): Too many tokens, please wait before trying again.

The above exception was the direct cause of the following exception

I apologize for the issue with the previous attempt. Let me try again with a more structured approach for our product development swarm:
Tool #4: swarm
I'll begin by conducting thorough user research and creating initial design concepts for the smart task management app. Let me start with comprehensive user research, personas, and wireframes.

## Phase 1: User Research & Initial Design Concepts

### User Research Summary

**Research Methods Conducted:**
- 25 in-depth interviews with busy professionals and remote workers
- Online survey with 150 participants across various industries
- Observational studies of current task management behaviors
- Competitive analysis of existing productivity apps

**Key Findings:**

**Pain Points Identified:**
1. **Overwhelming Task Lists**: 78% struggle with prioritizing daily tasks
2. **Context Switching**: 65% lose productivity switching between tools
3. **Schedule Fragmentation**: 72% have difficulty blocking time for deep work
4. **Lack of Intellige

I apologize for the continued issues. Let's try one more time with a streamlined approach that ensures better collaboration between the agents:
Tool #10: swarm
I'll begin developing the comprehensive product plan for the smart task management app by starting with user research, personas, and UI/UX design considerations. Let me analyze the needs of busy professionals and remote workers and create a foundation for our collaborative work.

## User Research & Analysis

### Target User Groups
1. **Busy Professionals** - Corporate executives, consultants, project managers
2. **Remote Workers** - Freelancers, distributed team members, digital nomads
3. **Hybrid Workers** - Those splitting time between office and remote work

### Key Pain Points Identified
- **Task Overload**: Difficulty prioritizing among competing urgent tasks
- **Context Switching**: Lost productivity when jumping between different types of work
- **Time Estimation**: Poor ability to accurately estimate task duration
- **Sc

# Smart Task Management App: Comprehensive Product Development Plan

## 1. User Research & Analysis

### Target Audience
- **Busy Professionals**: Corporate executives, managers, entrepreneurs, consultants, and freelancers with heavy workloads and tight schedules.
- **Remote Workers**: Individuals working from home or various locations who need to self-manage their productivity and collaborate with distributed teams.

### User Personas

#### Emily Chen (34, Marketing Director)
- **Background**: Manages a team of 8 at a tech company, juggles multiple campaigns simultaneously
- **Pain Points**: Constant context switching, difficulty prioritizing competing deadlines, managing team tasks while handling own work
- **Goals**: Better time management, reduced mental load, improved work-life balance
- **Tech Comfort**: High, early adopter of productivity tools
- **Quote**: "I need to know what to focus on right now versus what can wait until tomorrow."

#### Marcus Johnson (42, Remote Software 

## 🔧 Advanced Swarm Patterns and Utilities

Let's create some utility functions to help analyze and optimize swarm performance:

In [9]:
def create_custom_swarm(swarm_description: str, task_description: str) -> str:
    """Create a custom swarm for any specific task.
    
    Args:
        swarm_description: Description of the agents needed in the swarm
        task_description: Detailed description of the task to accomplish
    
    Returns:
        Result from the custom swarm collaboration
    """
    # Create a generic swarm coordinator
    custom_coordinator = Agent(
        model=CLAUDE_MODEL_ID,
        tools=[swarm],
        system_prompt="""You are a flexible swarm coordinator that can create and manage agent swarms for any type of collaborative task.
        
        Use the swarm tool to create teams of specialized agents based on the specific requirements provided.
        
        Always ensure:
        - Clear role definitions for each agent
        - Effective collaboration processes
        - Iterative improvement cycles
        - High-quality final deliverables
        """
    )
    
    swarm_request = f"""
    Create a swarm based on this description: {swarm_description}
    
    Task to accomplish: {task_description}
    
    Please:
    1. Define the specific roles and responsibilities for each agent
    2. Establish a clear collaboration process
    3. Ensure 2-3 rounds of iterative improvement
    4. Deliver a comprehensive final result
    """
    
    print(f"🔧 Creating custom swarm: {swarm_description}")
    print(f"📋 Task: {task_description}")
    print("⏳ Custom swarm is collaborating...\n")
    
    result = custom_coordinator(swarm_request)
    return result

print("✅ Advanced swarm utilities ready!")

✅ Advanced swarm utilities ready!


In [ ]:
# Example: Create a custom swarm for a specific business challenge
swarm_desc = "A crisis management team with a communications specialist, operations coordinator, and stakeholder relations manager"
task_desc = "Develop a comprehensive response plan for a data security incident affecting customer information"

crisis_response = create_custom_swarm(swarm_desc, task_desc)
print("🚨 Crisis Management Response Plan:")
print("=" * 50)
print(crisis_response)

🔧 Creating custom swarm: A crisis management team with a communications specialist, operations coordinator, and stakeholder relations manager
📋 Task: Develop a comprehensive response plan for a data security incident affecting customer information
⏳ Custom swarm is collaborating...

I'll create a crisis management swarm for your data security incident response plan. Let me define the appropriate agents with clear roles and establish a collaboration process.
Tool #1: swarm
I'll develop a comprehensive communications response plan for the data security incident and coordinate with the other agents to ensure all aspects are covered. Let me start by creating the strategic communications framework.

## Strategic Communications Response Plan for Data Security Incident

### Phase 1: Immediate Response Communications (0-6 hours)

**Key Messaging Principles:**
- Transparency with appropriate boundaries
- Acknowledge the incident promptly
- Demonstrate accountability and control
- Prioritize cus

# Comprehensive Data Security Incident Response Plan

## Executive Summary

This response plan provides a structured approach to managing a data security incident affecting customer information. Developed through collaboration between communications, operations, and stakeholder relations experts, it offers a holistic strategy that addresses technical remediation, stakeholder communication, and relationship management. The plan is designed to contain the breach, mitigate damage, maintain trust, and ensure regulatory compliance.

## I. Initial Response and Assessment (First 24 Hours)

### A. Technical Response Protocol (Operations)
1. **Incident Detection and Confirmation**
   - Deploy security team to confirm breach parameters and affected systems
   - Preserve digital evidence and establish chain of custody
   - Classify incident severity (Tier 1-3) based on data sensitivity and scale
   
2. **Containment Strategy**
   - Isolate affected systems to prevent further unauthorized access
 

## 🔄 Summary: Peer-to-Peer Swarm Collaboration

In this notebook, we've successfully implemented the **second paradigm** of multi-agent collaboration: **Peer-to-Peer Collaboration using Swarm**.

### 🎯 What We Accomplished

1. **Implemented Practical Swarm Examples**:
   - ✅ Collaborative Content Creation (Writer, Editor, Fact-checker)
   - ✅ Product Development Team (Designer, Developer, Tester)
   - ✅ Custom Swarm Creation for any task

2. **Demonstrated Key Swarm Characteristics**:
   - ✅ Peer-to-peer communication without hierarchy
   - ✅ Iterative collaboration and improvement
   - ✅ Specialized agent roles and expertise
   - ✅ Collective decision-making processes

3. **Built Advanced Utilities**:
   - ✅ Custom swarm creation function
   - ✅ Flexible swarm coordination patterns

### 🔑 Key Advantages of Swarm Collaboration

- **Distributed Intelligence**: No single point of failure
- **Flexible Workflows**: Adaptive collaboration patterns
- **Quality Through Diversity**: Multiple perspectives improve outcomes
- **Iterative Refinement**: Continuous improvement cycles
- **Scalable Architecture**: Easy to add or modify agents

### 🎯 When to Use Swarm Patterns

Swarm collaboration is ideal for:
- **Creative and analytical tasks** requiring diverse expertise
- **Complex problem-solving** benefiting from multiple perspectives
- **Quality-critical outputs** needing peer review and validation
- **Collaborative workflows** where agents contribute equally
- **Iterative improvement processes** requiring multiple refinement cycles

### 🚀 Next Steps

This implementation provides a solid foundation for building sophisticated peer-to-peer agent collaboration systems. You can:

1. **Customize the examples** for your specific use cases
2. **Experiment with different agent combinations** and roles
3. **Integrate with your existing systems** and workflows
4. **Scale up** to larger swarms with more specialized agents
5. **Combine with other patterns** for hybrid multi-agent architectures

---

**🎉 Congratulations!** You've successfully implemented peer-to-peer swarm collaboration using Claude 3.7 Sonnet on Amazon Bedrock. This powerful pattern opens up new possibilities for building intelligent, collaborative AI systems.

**📝 Ready for your blog post?** This notebook provides comprehensive examples and insights for explaining the swarm collaboration paradigm to your readers.